# Agenda

1. What are decorators, and how do they work?
2. Writing a simple timing decorator
3. Using the outer function for storage

https://github.com/reuven

https://github.com/reuven/pycontw-2024-decorators%202024-PyConTW-decorators.ipynb

# DRY -- don't repeat yourself



In [1]:
def a():
    return f'a!\n'

def b():
    return f'b!\n'

print(a())
print(b())

a!

b!



In [2]:
# Fix #1: Add the lines with variables and explicit interpolation

lines = '-' * 60 + '\n'

def a():
    return f'{lines}a!\n{lines}'

def b():
    return f'{lines}b!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [3]:
# Fix #2: Instead of calling our function directly,
# we will pass our function to another function which
# will include the lines

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'a!\n'

def b():
    return f'b!\n'

print(with_lines(a))
print(with_lines(b))

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [4]:
# Fix #3: Have with_lines return a new function,
# not a string. Calling this returned function
# will then return the string we want

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'a!\n'
with_lines_a = with_lines(a)

def b():
    return f'b!\n'
with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [5]:
# Fix #4: Instead of assigning to with_lines_a,
# just assign to a.  Same for with_lines_b

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'a!\n'
a = with_lines(a)

def b():
    return f'b!\n'
b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [6]:
# Fix #5: Use decorator syntax

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines
def a():
    return f'a!\n'
# a = with_lines(a)

@with_lines
def b():
    return f'b!\n'
# b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



# Summary of a decorator

1. It's a function (the decorator, what we mention with @)
2. That takes a function as an argument (the decorated function)
3. And returns a function as a result (the inner function, called wrapper)
4. That returned function (wrapper) replaces the orignal (decorated one)

# Who cares?

We now have a mechanism for hijacking a function (a) when it is defined and (b) when it is run, and we can replace either one of these stages with whatever we want.

1. Replace a function if permissions are wrong
2. Change the inputs
3. Change the outputs
4. Log information to another place
5. 

In [11]:
# let's add a new function to be decorated

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper(*args):  # take any number of positional args
        return f'{lines}{func(*args)}{lines}'  # unroll the args tuple into arguments
    return wrapper

@with_lines
def a():
    return f'a!\n'

@with_lines
def b():
    return f'b!\n'

@with_lines
def add(first, second):
    return f'{first} + {second} = {first+second}\n'
    
print(a())
print(b())
print(add(3, 5))

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------

------------------------------------------------------------
3 + 5 = 8
------------------------------------------------------------



# Exercise

Write a decorator that times how long it takes for a function to run. We'll call it `timefunc`. When we apply it to a function, the function runs normally and returns its normal result.

However, we also calculate how much time it took for the function to run and write that information to a file, `timing.txt`.

You can get the current Unix time (number of seconds since Jan 1, 1970) with `time.time()`.

If you need a slow function or two, you can use something like this:

```python
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second
```

    

In [16]:
import random
import time

def timefunc(func):   # outer function / decorator, runs once per decoration
    def wrapper(*args):   # inner function / called instead of our decorated func
        start_time = time.time()
        value = func(*args)
        total_time = time.time() - start_time

        # store the timing info to a file
        with open('timing.txt', 'a') as f:
            f.write(f'{func.__name__}\t{start_time}\t{total_time}\n')
        
        return value
    return wrapper   # don't forget this!

@timefunc
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@timefunc
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(2, 3))
print(slow_mul(3, 4))
print(slow_add(5, 6))
print(slow_mul(7, 8))

5
12
11
56


In [15]:
!cat timing.txt

slow_add	1726975475.8533509	1.9311904907226562e-05
slow_mul	1726975475.854787	1.4781951904296875e-05
slow_add	1726975475.85555	2.004378080368042
slow_mul	1726975477.860867	3.0010697841644287


# Caching

There is a mechanism called `memoization` in which we look at the arguments to a function. If we have seen the arguments before, we don't actually run the function. Rather, we retrieve the previously returned value for those arguments.

We can do this pretty easily with a decorator!



In [1]:
import random
import time


def memoize(func):   # outer function / decorator, runs once per decoration
    cache = {}
    def wrapper(*args):   # inner function / called instead of our decorated func
        # check args, which is a tuple -- is it in our dict?
        # if not, we run the function and store/cache the result in the dict
        
        if args not in cache:
            print(f'\tRunning {func.__name__} with {args}, and caching the result')
            cache[args] = func(*args)  # run the function, and cache the result
        else:
            print(f'\t{args} was cached; using that value')

        # now I know that the result is in the cache
        return cache[args]

    return wrapper   # don't forget this!

@memoize
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@memoize
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(2, 3))
print(slow_mul(2, 3))
print(slow_add(2, 3))
print(slow_mul(2, 3))

	Running slow_add with (2, 3), and caching the result
5
	Running slow_mul with (2, 3), and caching the result
6
	(2, 3) was cached; using that value
5
	(2, 3) was cached; using that value
6


In [3]:
slow_mul.__code__.co_freevars

('cache', 'func')

# Exercise: `once_per_minute`

Write a decorator, `once_per_minute`, that when applied to a function, raises an exception when the function is called twice within a 60-second period.




In [7]:
import random
import time

class CalledTooSoonError(Exception):
    pass

def once_per_minute(func):   # outer function / decorator, runs once per decoration
    last_called_at = 0

    def wrapper(*args):   # inner function / called instead of our decorated func
        nonlocal last_called_at  # this means: we want to assign to this variable in the enclosing func
        current_time = time.time()
        if current_time - last_called_at < 60:  # has <60 secs passed?
            raise CalledTooSoonError('Too soon!')

        # otherwise...
        last_called_at = current_time
        return func(*args)

    return wrapper   # don't forget this!

@once_per_minute
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@once_per_minute
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(2, 3))
print(slow_mul(2, 3))
print(slow_add(2, 3))
print(slow_mul(2, 3))

5
6


CalledTooSoonError: Too soon!

In [8]:
help(slow_add)

Help on function wrapper in module __main__:

wrapper(*args)



In [9]:
help(slow_mul)

Help on function wrapper in module __main__:

wrapper(*args)



In [ ]:
# I want to take the decorated function's signature and put it on the
# wrapper that we used to replace it

import random
import time


class CalledTooSoonError(Exception):
    pass

def once_per_minute(func):   # outer function / decorator, runs once per decoration
    last_called_at = 0

    def wrapper(*args):   # inner function / called instead of our decorated func
        nonlocal last_called_at  # this means: we want to assign to this variable in the enclosing func
        current_time = time.time()
        if current_time - last_called_at < 60:  # has <60 secs passed?
            raise CalledTooSoonError('Too soon!')

        # otherwise...
        last_called_at = current_time
        return func(*args)

    return wrapper   # don't forget this!

@once_per_minute
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@once_per_minute
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(2, 3))
print(slow_mul(2, 3))
print(slow_add(2, 3))
print(slow_mul(2, 3))